# Transform Phenotypic Data into a Common Format
This script accesses the extracted data in their original format and applys a mapping to harmonize the data into a common tabular format.

In [34]:
import os
import json
import glob

import numpy as np
import pandas as pd

In [10]:
# Path to repositories to pull raw data out of.
curdir = os.path.abspath(os.path.curdir)
repos = ['fcp-indi']
extract_paths = [os.path.join(curdir, repo) for repo in repos]
# Path to where the tranformed projects will go.
xfm_dir = os.path.join(curdir, 'metadata')
if not os.path.exists(xfm_dir):
    os.mkdir(xfm_dir)
# Project dirs.
extract_dirs = list()
for extract_path in extract_paths:
    project_paths = [os.path.join(extract_path, project) for project in os.listdir(extract_path) if '.' not in project]
    extract_dirs.extend(project_paths)

In [122]:
def apply_mapping(mapping, df_list):
    """
    Reads a dictionary mapping and list of dataframe, then merges the
    dataframes and convers the column name and values into a common
    format. Note: the dataframes should have a common structure.
    
    Example Mapping Structure
    =========================
    {  
        "DX_GROUP":
      {
        "element": "diagnosis",
        "type": "category",
        "1": "autism",
        "2": "control"
      }
    }
    """
    results = list()
    csv = pd.concat(df_list)
    csv.reset_index(drop=True, inplace=True)
    for col, elem in mapping.iteritems():
        # Use get in case the col is missing/static.
        series = csv.get(col)
        # Categories are mapped to common values.
        if elem.get('type') == 'str':
            result = series.apply(lambda x: elem.get(str(x)))
        # Values parsed as numbers are checked for any mappings (e.g., -999 == NaN).
        elif elem.get('type') == 'float':
            result = series.apply(lambda x: np.NaN if str(x) in elem.keys() else x)
        # Used to create a column of all the same value.
        elif elem.get('type') == 'static':
            val = [elem.get('value')] * csv.shape[0]
            result = pd.Series(val, index=csv.index)
        else:
            try:
                # Handle IDs being read as float.
                result = series.apply(lambda x: str(int(x)) if pd.notnull(x) else x)
            except ValueError as e:
                result = series
        # Concat all csv in a extract dir into one dataframe.        
        df = pd.DataFrame()
        df[elem.get('element')] = result
        results.append(df)
        concat = pd.concat(results, axis=1)
    # Merge any columns with duplicate names to fil in NaN from concat step.
    return concat.groupby(level=0, axis=1).first()

In [123]:
# Read a mapping.json file from each sub directory to process.
for extract_dir in extract_dirs:
    df_list = list()
    mapping = dict()
    project_name = extract_dir.split('/')[-1]
    # All files must use the same data dictionary in a given directory.
    extract_files = [os.path.join(extract_dir, i) for i in os.listdir(extract_dir) if i not in ['mapping.json']]
    mapping_file = os.path.join(extract_dir, 'mapping.json')
    if os.path.exists(mapping_file):
        with open(mapping_file, 'rb') as fi:
            mapping.update(json.load(fi))
        ext_type = dict(csv=',', tsv='\t')
        # Grab dtype for parsing csv
        dtype = {k: v.get('type') for k, v in mapping.iteritems() if v.get('type') in ['str', 'int', 'float']}
        dtype.update({k: 'str' for k, v in mapping.iteritems() if v.get('type') in ['static']})
        # Dealing with NaNs.
        na_values = ['NoPhenotypicData', '#']
        for extract_file in extract_files:
            # Process each file.
            ext = extract_file.split('.')[-1]
            sep_type = ext_type.get(ext)
            df = pd.read_csv(extract_file, sep=sep_type, dtype=dtype, na_values=na_values)
            df_list.append(df)
        xfm = apply_mapping(mapping, df_list)
        pheno_file = ''.join([project_name, '_', 'phenotype.csv'])
        pheno_path = os.path.join(xfm_dir, pheno_file)
        xfm.to_csv(pheno_path, index=False)

In [118]:
# Merge all the transformed files into a single frame
meta_files = glob.glob(xfm_dir + '/*.csv')
meta_list = list()
for meta_file in meta_files:
    df = pd.read_csv(meta_file)
    meta_list.append(df)
meta_df = pd.concat(meta_list).set_index(['project', 'session_id', 'participant_id'], drop=False)
meta_df.head()

age diagnosis dsm_iv_tr  \
project          session_id participant_id                              
abide_initiative 1          50002           16.77    autism    autism   
                            50003           24.45    autism    autism   
                            50004           19.09    autism    autism   
                            50005           13.73    autism    autism   
                            50006           13.37    autism    autism   

                                            full iq    handedness  \
project          session_id participant_id                          
abide_initiative 1          50002             103.0  ambidextrous   
                            50003             124.0  right handed   
                            50004             113.0  right handed   
                            50005             119.0  right handed   
                            50006             109.0   left handed   

                                           participant_id  performance iq  \
project          session_id participant_id                                  
abide_initiative 1          50002                   50002            89.0   
                            50003                   50003           115.0   
                            50004                   50004           117.0   
                            50005                   50005           118.0   
                            50006                   50006           119.0   

                                                     project session_id  \
project          session_id participant_id                                
abide_initiative 1          50002           abide_initiative          1   
                            50003           abide_initiative          1   
                            50004           abide_initiative          1   
                            50005           abide_initiative          1   
                            50006           abide_initiative          1   

                                               sex site_id       species  \
project          session_id participant_id                                 
abide_initiative 1          50002             Male    PITT  homo-sapiens   
                            50003             Male    PITT  homo-sapiens   
                            50004             Male    PITT  homo-sapiens   
                            50005           Female    PITT  homo-sapiens   
                            50006             Male    PITT  homo-sapiens   

                                            verbal iq  
project          session_id participant_id             
abide_initiative 1          50002               116.0  
                            50003               128.0  
                            50004               108.0  
                            50005               117.0  
                            50006                99.0

In [90]:
# Load the CSV with MRI URLs
mri_path = os.path.abspath('clean-csv/all-session.csv')
mri = pd.read_csv(mri_path).set_index(['project', 'session_id', 'participant_id'], drop=False)
mri.head()

t1_url  \
project session_id participant_id                                                      
acpi    1          0028031         https://s3.amazonaws.com/fcp-indi/data/Project...   
                   0028032         https://s3.amazonaws.com/fcp-indi/data/Project...   
                   0028033         https://s3.amazonaws.com/fcp-indi/data/Project...   
                   0028034         https://s3.amazonaws.com/fcp-indi/data/Project...   
                   0028035         https://s3.amazonaws.com/fcp-indi/data/Project...   

                                  participant_id project session_id  \
project session_id participant_id                                     
acpi    1          0028031               0028031    acpi          1   
                   0028032               0028032    acpi          1   
                   0028033               0028033    acpi          1   
                   0028034               0028034    acpi          1   
                   0028035               0028035    acpi          1   

                                   session_count  
project session_id participant_id                 
acpi    1          0028031                     1  
                   0028032                     1  
                   0028033                     1  
                   0028034                     1  
                   0028035                     1

In [124]:
mri.project.unique()

array(['acpi', 'adhd200', 'corr', 'indi', 'rocklandsample'], dtype=object)